<a href="https://colab.research.google.com/github/JoseAguirre-10/LogosFCFM/blob/master/P2_Tarea_2_Opti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 28.0 MB/s eta 0:00:00


In [17]:
import numpy as np
from gurobipy import*

In [149]:
#para esta pregunta se nos dan ciertos coeficientes a los cuales se les asignaran un valor
#Primero voy a plantear nuevamente el problema de optimización

#Definimos los datos que nos entrega el problema.
betha = [0,100,120,130,140,150,200,0] #Esto representa el costo de almacenaje



c = np.array([[0,10,25,30,0,0,20,0],[0,0,0,0,15,15,0,0],[0,0,0,0,0,15,0,0],
 [0,0,0,0,0,0,10,0],[0,0,0,0,0,0,0,25],[0,0,0,0,0,0,0,80],[0,0,0,0,0,0,0,25],  #representa el costo de ir desdela bodega i-esima hasta la j-esima, cij = 0 representa que esa ruta no existe
  [0,0,0,0,0,0,0,0]])

r = np.array([[0,500,350,500,0,0,600,0],[0,0,0,0,250,350,0,0],[0,0,0,0,0,350,0,0],
 [0,0,0,0,0,0,500,0],[0,0,0,0,0,0,0,800],[0,0,0,0,0,0,0,500],[0,0,0,0,0,0,0,500], #Representa cuanto se piuede transportar en la ruta desde i hasta j. Rij = 0 significa que esa ruta no está definida
  [0,0,0,0,0,0,0,0]])



In [159]:
#planteamos el problema de optimización
m = Model('Penalización')
n = 8

#definimos las variables, en este caso tenemos un problema de una sola variable, el cual no es lineal

x = m.addVars(n,n,vtype=GRB.CONTINUOUS, name = 'Cantidad_de_elemntos_por_bodega')
#restricciones:

#Se define el punto de salida
m.addConstr(x[0,0] == 1000, name = 'salida')
#se deben bloquear las rutas no existentes
for i in range(n):
    for j in range(n):
        if R[i, j] <= 0:  # Bloquear rutas no válidas
            m.addConstr(x[i, j] == 0, name=f'Bloqueo_{i}_{j}')
        else:
            m.addConstr(x[i, j] <= R[i, j], name=f'Capacidad_{i}_{j}')

#
m.addConstrs((quicksum(x[i,j] for i in range(n)) == 1000 for j in range (n)), name = 'Suma columna i-esima en X')

m.addConstrs(((x[i,j] -float(r[i,j])) <=0 for i in range (n) for j in range (n) if float(r[i,j])>0), name = 'Carga ruta ij')
m.addConstrs((x[i,j] >= 0 for i in range(n) for j in range(n)), name = 'restriccion')
m.setObjective(
    quicksum(float(C[i, j]) * x[i, j] * x[i, j] for i in range(n) for j in range(n)) +
    quicksum(float(betha[i]) * quicksum(x[i, j] for j in range(n)) for i in range(n)),
    GRB.MINIMIZE)


m.Params.NonConvex = 2

m.optimize()

print(f"Status: {m.Status}")
if m.Status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            val = x[i, j].X
            if val > 1e-6:
                print(f"x[{i},{j}] = {val:.2f}")
else:
    print("No se encontró solución óptima.")

Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
NonConvex  2

Optimize a model with 137 rows, 64 columns and 193 nonzeros
Model fingerprint: 0xf3388ae1
Model has 11 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  QObjective range [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 127 rows and 1 columns
Presolve time: 0.01s

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Model is infeasible or unbounded
Status: 4
No se encontró solución óptima.
